## Example to run gvec using python, and visualize the result


**Note:**

**This ipython notebook needs prior installation of the gvec python package**

**within a virtual environment, which then should be chosen as kernel!**

In [ ]:
import os
from pathlib import Path
import contextlib

import numpy as np
import matplotlib.pyplot as plt

# OMP number of threads for gvec run needs to be before import of gvec
os.environ["OMP_NUM_THREADS"] = "4"
# needs `pip install` of gvec in virtual environment, and to be run in that environment!!!
import gvec  # using run & modifying the parameters & postprocessing

In [ ]:
# helper
@contextlib.contextmanager
def chdir(path: Path | str):
    """
    Contextmanager to change the current working directory.

    Using a context has the benefit of automatically changing back to the original directory when the context is exited, even if an exception is raised.
    """
    path = Path(path)
    old_dir = Path(os.getcwd())

    os.chdir(path)
    yield
    os.chdir(old_dir)

### Run gvec from a parameter file 

In this example, the input parameter file in this directory `parameter.ini` is used as a template. 

The input contains the boundary description as Fourier modes (here a simple circular tokamak), the profiles (iota and pressure) and discretization parameters.

Here, we will modify some parameters and then run the equilibrium solver gvec, in a sub-directory.


In [ ]:
template = "parameter.ini"

# collect all parameters in a dictionary:
params = {}
# polynomial coefficients of the iota (rotational transform) and pressure profile
iota_coefs = [0.523, 0.625]  # c_0*s + c_1
pres_coefs = [-1.36878012275e-02, 0.02]
params["iota_coefs"] = "(/" + ", ".join(map(str, iota_coefs[::-1])) + "/)"
params["pres_coefs"] = "(/" + ", ".join(map(str, pres_coefs[::-1])) + "/)"
# number of radial B-spline elements
params["sgrid_nElems"] = 5
# maximum number of iterations
params["maxiter"] = 1000
# First fourier mode of boundary shape, X1=R, X2=Z
params["X1_b_cos"] = {(1, 0): 0.9}
params["X2_b_sin"] = {(1, 0): 1.1}


# create a run directory
runpath = Path(f"run_{1:02d}")
if not runpath.exists():
    runpath.mkdir()
    print(f"created run directory {runpath}")

# copy template and modify parameters:
gvec.util.adapt_parameter_file(template, runpath / "parameter.ini", **params)

# run GVEC simulation
with chdir(runpath):
    gvec.run("parameter.ini", stdout_path="stdout.txt")

### Post-processing: evaluate the equilibrium

Here, we load the final state file written in the gvec run, together with the parameterfile used. 
Then we can choose a discrete set of 1d points in radial direction `rho` (proportional to the square-root of the magnetic flux) and poloidal direction `theta` and toroidal direction `zeta`. Either by specifying an array, or just the number of points. 

As this example is a tokamak, only one point in toroidal direction is used.

In [ ]:
statefile = sorted(runpath.glob("*State*.dat"))[-1]
with gvec.State(runpath / "parameter.ini", statefile) as state:
    rho = np.linspace(0, 1, 20)  # radial visualization points
    theta = np.linspace(0, 2 * np.pi, 50)  # poloidal visualization points
    ev = gvec.Evaluations(rho=rho, theta=theta, zeta=1, state=state)
    state.compute(ev, "X1", "X2", "LA", "iota", "p")

### Visualize the result

Now, lets visualize the 1D profiles (input quantities)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(8, 4))

ax[0].plot(ev.rho**2, ev.iota)
ax[0].set(xlabel="$\\rho^2\sim$ tor. flux", title="iota profile")
ax[1].plot(ev.rho**2, ev.p)
ax[1].set(xlabel="$\\rho^2\sim$ tor. flux", title="pressure profile");


And visualizte the 2D cross-section of the example.  Note that the contours of the straight-field line angle $\theta^* =\theta +\lambda(\theta,\zeta)$  are shown.

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 4))

R = ev.X1[:, :, 0]
Z = ev.X2[:, :, 0]
rho_vis = R * 0 + ev.rho
theta_vis = R * 0 + ev.theta
thetastar_vis = ev.LA[:, :, 0] + ev.theta
p_vis = R * 0 + ev.p
rho_levels_vis = np.linspace(0, 1 - 1e-10, 11)
theta_levels_vis = np.linspace(0, 2 * np.pi, 16, endpoint=False)
c = ax.contourf(R, Z, p_vis, alpha=0.75)
fig.colorbar(c, ax=ax, label="pressure")
ax.contour(R, Z, rho_vis, rho_levels_vis, colors="black")
ax.contour(R, Z, thetastar_vis, theta_levels_vis, colors="red")
ax.set(
    xlabel="$R$",
    ylabel="$Z$",
    title="equilibrium solution, cross-section",
    aspect="equal",
    xlim=[0.8 * np.amin(R), 1.2 * np.amax(R)],
    ylim=[1.1 * np.amin(Z), 1.1 * np.amax(Z)],
)
ax.legend(
    handles=[
        plt.Line2D([0], [0], color="black", label="$\\rho=$const."),
        plt.Line2D([0], [0], color="red", label="$\\theta^*=$const"),
    ]
)


R_axis = R[0, 0].item()
Z_axis = Z[0, 0].item()

print(f"R,Z position of magnetic axis={R_axis,Z_axis}")